# CHAPTER 4
# Pandas Basics

Pandas stands for “Python Data Analysis Library”. It contains data structures and data manipulation tools designed to make data cleaning and analysis fast and easy in Python. pandas is often used in tandem with numerical computing tools like NumPy and SciPy, analytical libraries like statsmodels and scikit-learn, and data visualization libraries like matplotlib. pandas adopts significant parts of NumPy’s idiomatic style of array-based computing, especially array-based functions and a preference for data processing without for loops. While pandas adopts many coding idioms from NumPy, the biggest difference is that pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast, is best suited for working with homogeneous numerical array data. 

## 4.1 Introduction to pandas Data Structures 

To get started with pandas, you will need to get comfortable with its two workhorse data structures: Series and DataFrame. While they are not a universal solution for every problem, they provide a solid, easy-to-use basis for most applications. 

### 4.1.1 Series 

A Series is a **one-dimensional** array-like object containing a sequence of values (of similar types to NumPy types) and an associated array of data labels, called its index. The simplest Series is formed from only an array of data:


In [255]:
import pandas as pd

obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Since we did not specify an index for the data, a default one consisting of the integers 0 through N - 1 (where N is the length of the data) is created. You can get the array representation and index object of the Series via its values and index attributes, respectively:

In [256]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [257]:
obj.index

RangeIndex(start=0, stop=4, step=1)

Often it will be desirable to create a Series with an index identifying each data point with a label:

In [258]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [259]:
print(obj2.index)
print()
print(obj2.values)

Index(['d', 'b', 'a', 'c'], dtype='object')

[ 4  7 -5  3]


Compared with NumPy arrays, you can use labels in the index when selecting single values or a set of values:

In [260]:
print(obj2['a'])

-5


In [261]:
obj2['d']=6
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [262]:
 obj2[['c', 'a', 'd']] 

c    3
a   -5
d    6
dtype: int64

Here ['c', 'a', 'd'] is interpreted as a list of indices, even though it contains strings instead of integers. 

Using NumPy functions or NumPy-like operations, such as filtering with a boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [263]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [264]:
obj2[obj2 > 0] 

d    6
b    7
c    3
dtype: int64

In [265]:
 obj2 * 2 

d    12
b    14
a   -10
c     6
dtype: int64

In [266]:
import numpy as np

np.exp(obj2) 

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Another way to think about a Series is as a fixed-length, ordered dict, as it is a mapping of index values to data values. It can be used in many contexts where you might use a dict:

In [267]:
'b' in obj2 

True

In [268]:
'e' in obj2 

False

Should you have data contained in a Python dict, you can create a Series from it by passing the dict:

In [269]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = pd.Series(sdata)
obj3 

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

When you are only passing a dict, the index in the resulting Series will have the dict’s keys in sorted order. You can override this by passing the dict keys in the order you want them to appear in the resulting Series:

In [270]:
states = ['California', 'Ohio', 'Oregon', 'Texas'] # Change Ohio to California
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

Here, three values found in *sdata* were placed in the appropriate locations, but since no value for 'California' was found, it appears as NaN (not a number), which is considered in pandas to mark missing or NA values. Since 'Utah' was not included in states, it is excluded from the resulting object. 

We will use the terms “missing” or “NA” interchangeably to refer to missing data. The **isnull** and **notnull** functions in pandas should be used to detect missing data:


In [271]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [272]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

Series also has these as instance methods:

In [273]:
obj4.isnull() 

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [274]:
print(obj3)

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64


In [275]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [276]:
obj3 + obj4 

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Both the Series object itself and its index have a **name** attribute, which integrates with other key areas of pandas functionality:

In [277]:
obj4.name = 'POPULATION'
obj4.index.name = 'STATE'
obj4 

STATE
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: POPULATION, dtype: float64

A Series’s index can be altered in-place by assignment:

In [278]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [279]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### 4.1.2 DataFrame 

A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc.). The DataFrame has both a row and column index; it can be thought of as a dict of Series all sharing the same index. Under the hood, the data is stored as one or more two-dimensional blocks rather than a list, dict, or some other collection of one-dimensional arrays.

<br>
<img src="F1.png", style="width: 500px";>

There are many ways to construct a DataFrame, though one of the most common is from a dict of equal-length lists or NumPy arrays:

    data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],        
            'year': [2000, 2001, 2002, 2001, 2002, 2003],        
            'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]} 
    
    frame = pd.DataFrame(data) 
    
The resulting DataFrame will have its index assigned automatically as with Series, and the columns are placed in sorted order:


In [280]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],       
        'year': [2000, 2001, 2002, 2001, 2002, 2003],       
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]} 

print(data)

frame = pd.DataFrame(data) 

frame

{'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'], 'year': [2000, 2001, 2002, 2001, 2002, 2003], 'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}


,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


For large DataFrames, the head method selects only the first five rows:

In [281]:
frame.head() 

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


If you specify a sequence of columns, the DataFrame’s columns will be arranged in that order:

In [282]:
pd.DataFrame(data, columns=['year', 'state', 'pop']) 

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


If you pass a column that isn’t contained in the dict, it will appear with missing values in the result:

In [283]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'population'], 
                       index=['one', 'two', 'three', 'four','five', 'six'])

frame2

,year,state,pop,population
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [284]:
frame2.columns

Index(['year', 'state', 'pop', 'population'], dtype='object')

In [285]:
frame2.index

Index(['one', 'two', 'three', 'four', 'five', 'six'], dtype='object')

A column in a DataFrame can be retrieved as a Series either by dict-like notation or by attribute:

In [286]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [287]:
frame2.state

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

*frame2[column]* works for any column name, but *frame2.column* only works when the column name is a valid Python variable name.

Note that the returned Series have the same index as the DataFrame, and their **name** attribute has been appropriately set. 

Rows can also be retrieved by position or name with the special **loc** attribute (much more on this later):


In [288]:
frame2.loc['three']

year          2002
state         Ohio
pop            3.6
population     NaN
Name: three, dtype: object

Columns can be modified by assignment. For example, the empty 'debt' column could be assigned a scalar value or an array of values:

In [289]:
frame2

,year,state,pop,population
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [290]:
frame2['debt'] = 16.5
frame2

,year,state,pop,population,debt
one,2000,Ohio,1.5,NaN,16.5
two,2001,Ohio,1.7,NaN,16.5
three,2002,Ohio,3.6,NaN,16.5
four,2001,Nevada,2.4,NaN,16.5
five,2002,Nevada,2.9,NaN,16.5
six,2003,Nevada,3.2,NaN,16.5


In [291]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,population,debt
one,2000,Ohio,1.5,NaN,0.0
two,2001,Ohio,1.7,NaN,1.0
three,2002,Ohio,3.6,NaN,2.0
four,2001,Nevada,2.4,NaN,3.0
five,2002,Nevada,2.9,NaN,4.0
six,2003,Nevada,3.2,NaN,5.0


When you are assigning lists or arrays to a column, the value’s length must match the length of the DataFrame. If you assign a Series, its labels will be realigned exactly to the DataFrame’s index, inserting missing values in any holes:


In [292]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
print(val)
print()
print(frame2)

frame2['debt'] = val
frame2 

two    -1.2
four   -1.5
five   -1.7
dtype: float64

       year   state  pop population  debt
one    2000    Ohio  1.5        NaN   0.0
two    2001    Ohio  1.7        NaN   1.0
three  2002    Ohio  3.6        NaN   2.0
four   2001  Nevada  2.4        NaN   3.0
five   2002  Nevada  2.9        NaN   4.0
six    2003  Nevada  3.2        NaN   5.0


,year,state,pop,population,debt
one,2000,Ohio,1.5,NaN,NaN
two,2001,Ohio,1.7,NaN,-1.2
three,2002,Ohio,3.6,NaN,NaN
four,2001,Nevada,2.4,NaN,-1.5
five,2002,Nevada,2.9,NaN,-1.7
six,2003,Nevada,3.2,NaN,NaN


Assigning a column that doesn’t exist will create a new column. The **del** keyword will delete columns as with a dict. As an example of **del**, we first add a new column of boolean values where the state column equals 'Ohio':


In [293]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,population,debt,eastern
one,2000,Ohio,1.5,NaN,NaN,True
two,2001,Ohio,1.7,NaN,-1.2,True
three,2002,Ohio,3.6,NaN,NaN,True
four,2001,Nevada,2.4,NaN,-1.5,False
five,2002,Nevada,2.9,NaN,-1.7,False
six,2003,Nevada,3.2,NaN,NaN,False


The **del** method can then be used to remove this column:

In [294]:
del frame2['population']
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


The column returned from indexing a DataFrame is a view on the underlying data, not a copy. Thus, any in-place modifications to the Series will be reflected in the DataFrame. The column can be explicitly copied with the Series’s copy method.

Another common form of data is a nested dict of dicts:

In [295]:
pop = {'Nevada': {2002: 2.9, 2001: 2.4},  
       'Ohio': {2001: 1.7,2000: 1.5,  2002: 3.6}} 

frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2002,2.9,3.6
2001,2.4,1.7
2000,NaN,1.5


You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [296]:
frame3.T

,2002,2001,2000
Nevada,2.9,2.4,NaN
Ohio,3.6,1.7,1.5


The keys in the inner dicts are combined and sorted to form the index in the result. This isn’t true if an explicit index is specified:

In [297]:
pop

{'Nevada': {2002: 2.9, 2001: 2.4}, 'Ohio': {2001: 1.7, 2000: 1.5, 2002: 3.6}}

In [298]:
 pd.DataFrame(pop, index=[2001, 2003, 2002])

,Nevada,Ohio
2001,2.4,1.7
2003,NaN,NaN
2002,2.9,3.6


Dicts of Series are treated in much the same way:

In [299]:
frame3

,Nevada,Ohio
2002,2.9,3.6
2001,2.4,1.7
2000,NaN,1.5


In [300]:
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}

pd.DataFrame(pdata) 

,Ohio,Nevada
2002,3.6,2.9
2001,1.7,2.4


For a complete list of things you can pass the DataFrame constructor, see Table 4-1. 

<br>
<center>Table 4.1: Possible data inputs to DataFrame constructor  </center>
<img src="Table4.1.jpg", style="width: 800px";>

If a DataFrame’s **index** and **columns** have their **name** attributes set, these will also be displayed:

In [301]:
frame3.index.name = 'Year'; frame3.columns.name = 'State'
frame3

State,Nevada,Ohio
Year,,
2002,2.9,3.6
2001,2.4,1.7
2000,NaN,1.5


As with Series, the values attribute returns the data contained in the DataFrame as a two-dimensional ndarray:

In [302]:
frame3.values

array([[2.9, 3.6],
       [2.4, 1.7],
       [nan, 1.5]])

If the DataFrame’s columns are different dtypes, the dtype of the values array will be chosen to accommodate all of the columns:

In [303]:
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [304]:
frame2.values

array([[2000, 'Ohio', 1.5, nan, True],
       [2001, 'Ohio', 1.7, -1.2, True],
       [2002, 'Ohio', 3.6, nan, True],
       [2001, 'Nevada', 2.4, -1.5, False],
       [2002, 'Nevada', 2.9, -1.7, False],
       [2003, 'Nevada', 3.2, nan, False]], dtype=object)

### 4.1.3 Index Objects 

pandas’s Index objects are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels you use when constructing a Series or DataFrame is internally converted to an Index:


In [305]:
obj = pd.Series(range(3), index=['a', 'b', 'b'])
print(obj)

index = obj.index
index 

a    0
b    1
b    2
dtype: int64


Index(['a', 'b', 'b'], dtype='object')

In [306]:
obj['b']

b    1
b    2
dtype: int64

In [307]:
index[1:]

Index(['b', 'b'], dtype='object')

Index objects are immutable and thus can’t be modified by the user:

In [308]:
#index[1]='d'

Immutability makes it safer to share Index objects among data structures:

In [309]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [310]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [311]:
#labels[1]=4

Some users will not often take advantage of the capabilities provided by indexes, but because some operations will yield results containing indexed data, it’s important to understand how they work.

In addition to being array-like, an Index also behaves like a fixed-size set:

In [312]:
frame3

State,Nevada,Ohio
Year,,
2002,2.9,3.6
2001,2.4,1.7
2000,NaN,1.5


In [313]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='State')

In [314]:
'Ohio' in frame3.columns 

True

In [315]:
2003 in frame3.index 

False

Unlike Python sets, a pandas Index can contain duplicate labels:

In [316]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels 

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

Selections with duplicate labels will select all occurrences of that label. 

Each Index has a number of methods and properties for set logic, which answer other common questions about the data it contains. Some useful ones are summarized in Table 4-2.

<br>
<center>Table 4.2: Some Index methods and properties </center>
<img src="Table4.2.jpg", style="width: 800px";>

In [317]:
df1 = pd.Index([17, 69, 33, 5, 0, 74, 0])
df2 = pd.Index([11, 16, 54, 58])

df1.append(df2)

Int64Index([17, 69, 33, 5, 0, 74, 0, 11, 16, 54, 58], dtype='int64')

In [318]:
df1.difference(df2)

Int64Index([0, 5, 17, 33, 69, 74], dtype='int64')

In [319]:
df1.intersection(df2)

Int64Index([], dtype='int64')

In [320]:
df1.union(df2)

Int64Index([0, 0, 5, 11, 16, 17, 33, 54, 58, 69, 74], dtype='int64')

In [321]:
df1.isin([17, 5])

array([ True, False, False,  True, False, False, False])

In [322]:
df2.delete([1])

Int64Index([11, 54, 58], dtype='int64')

In [323]:
df1.drop([69])

Int64Index([17, 33, 5, 0, 74, 0], dtype='int64')

In [324]:
df2.insert(2,3)

Int64Index([11, 16, 3, 54, 58], dtype='int64')

In [325]:
df2.unique()

Int64Index([11, 16, 54, 58], dtype='int64')